# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846848952674                   -0.70    4.8
  2   -7.852318718680       -2.26       -1.53    1.0
  3   -7.852646223678       -3.48       -2.52    3.5
  4   -7.852646678058       -6.34       -3.37    2.2
  5   -7.852646685859       -8.11       -4.71    1.0
  6   -7.852646686723       -9.06       -5.24    3.5
  7   -7.852646686730      -11.18       -6.22    1.8
  8   -7.852646686730      -12.78       -7.75    2.5
  9   -7.852646686730      -14.75       -7.77    2.5
 10   -7.852646686730   +  -14.75       -8.83    2.0
 11   -7.852646686730   +    -Inf       -9.58    2.0
 12   -7.852646686730   +  -15.05       -9.59    1.0
 13   -7.852646686730   +    -Inf       -8.80    1.0
 14   -7.852646686730   +  -14.57       -9.03    1.0
 15   -7.852646686730      -14.75       -9.87    1.0
 16   -7.852646686730      -14.35      -10.18    1.0
 17   -7.852646686730   +  -14.45      -10.64 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846821884008                   -0.70           5.0
  2   -7.852528511583       -2.24       -1.63   0.80    2.0
  3   -7.852637477166       -3.96       -2.71   0.80    1.0
  4   -7.852646470953       -5.05       -3.31   0.80    2.2
  5   -7.852646682492       -6.67       -4.16   0.80    1.8
  6   -7.852646686392       -8.41       -4.85   0.80    1.5
  7   -7.852646686723       -9.48       -5.77   0.80    1.8
  8   -7.852646686730      -11.16       -6.74   0.80    2.2
  9   -7.852646686730      -12.80       -7.60   0.80    2.5
 10   -7.852646686730      -14.75       -8.33   0.80    1.8
 11   -7.852646686730   +  -14.45       -8.70   0.80    2.0
 12   -7.852646686730      -14.75       -9.77   0.80    1.2
 13   -7.852646686730      -14.57       -9.98   0.80    2.5
 14   -7.852646686730   +  -14.57      -10.73   0.80    1.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.356134e+01     3.095864e+00
 * time: 0.7231161594390869
     1     9.094590e-01     1.638548e+00
 * time: 1.0266690254211426
     2    -1.099486e+00     1.922734e+00
 * time: 1.064223051071167
     3    -3.774758e+00     1.504480e+00
 * time: 1.1185650825500488
     4    -4.842988e+00     1.535267e+00
 * time: 1.177976131439209
     5    -6.628818e+00     8.216966e-01
 * time: 1.2426090240478516
     6    -7.247599e+00     3.160462e-01
 * time: 1.2974810600280762
     7    -7.498483e+00     2.072187e-01
 * time: 1.334348201751709
     8    -7.638024e+00     1.454395e-01
 * time: 1.3717470169067383
     9    -7.731929e+00     1.449287e-01
 * time: 1.4118330478668213
    10    -7.791794e+00     1.021915e-01
 * time: 1.450423002243042
    11    -7.825076e+00     6.738230e-02
 * time: 1.4879980087280273
    12    -7.842883e+00     3.553740e-02
 * time: 1.5247600078582764
    13    -7.849156e+00     2.692520e-02
 * time: 1.56383800506591

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846798660437                   -0.70    4.5
  2   -7.852300200319       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686597                   -2.54
  2   -7.852646686730       -9.88       -5.87
  3   -7.852646686730   +    -Inf      -11.77
  4   -7.852646686730      -14.75      -15.71


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.4140811510856365e-13
|ρ_newton - ρ_scfv| = 1.4407071430091798e-12
|ρ_newton - ρ_dm|   = 7.751055466463193e-10
